# Error Mitigation: Zero Noise Extrapolation

_This notebook contains experimental resultd for Zero Noise Extrapolation (ZNE)._

Zero Noise Extrapolation (ZNE) is one of most used techniques in quantum error mitigation to estimate any expectation value of an observable to noise less condition. In ZNE, we intentionally introduce noise in the quantuum circuit and measure the expectation value of the target obseavble at different noise level. Once we have the sufficient data, we extrapolate our data to zero-noise limit.

There are several stratigies for zero-noise extrapolation, and perhaps the most common one is Richardson extraapolattion technique. Here, we adopted the multivariate framework of Richardson extrapolation used in this paper.

In [18]:
"""
We have used the multivariate framework for Richardson extrapolation as discussed in the paper "Quantum error mitigation by layerwise Richardson extrapolation" by Vincent Russo and Andrea Mari (arXiv:2402.04000, 2024).

Parts of the following code are adapted from their notebook, which can be found at the following GitHub repository: https://github.com/unitaryfund/research/blob/main/lre/layerwise_richardson_extrapolation.ipynb.
"""

import numpy as np
from collections import Counter
import itertools


class ZeroNoiseExtrapolation:
    def __init__(self, dataPoints: list[tuple[float]], degree: int):
        """
        Initialize with a list of data points, each represented as a tuple of floats.
        """
        self.dataPoints = dataPoints
        self.degree = degree

        self.NoiseData = [tuple(point[:3]) for point in self.dataPoints]
        self.ExpectationVals = [point[-1] for point in dataPoints]
  
    def getRichardsonZNE(self):

        RichardsonZNEval = 0

        sampleMatrix = sample_matrix(sample_points = self.NoiseData, degree = self.degree) # type: ignore
        detA = np.linalg.det(sampleMatrix)

        matrices = generate_modified_matrices(sampleMatrix) # type: ignore

        for E, matrix in zip(self.ExpectationVals, matrices):
            RichardsonZNEval += E * (np.linalg.det(matrix)/detA)
        
        return RichardsonZNEval
    
    @staticmethod
    def get_monomials(n: int, d: int) -> list[str]:
        """
        Compute monomials of degree `d` in graded lexicographical order.
        """
        variables = [f"λ_{i}" for i in range(1, n + 1)]
        
        monomials = []
        for degree in range(d, -1, -1):
            # Generate combinations for the current degree
            combos = list(itertools.combinations_with_replacement(variables, degree))
            
            # Sort combinations lexicographically
            combos.sort()
            
            # Construct monomials from sorted combinations
            for combo in combos:
                monomial_parts = []
                counts = Counter(combo)
                # Ensure variables are processed in lexicographical order
                for var in sorted(counts.keys()):
                    count = counts[var]
                    if count > 1:
                        monomial_parts.append(f"{var}**{count}")
                    else:
                        monomial_parts.append(var)
                monomial = "*".join(monomial_parts)
                # Handle the case where degree is 0
                if not monomial:
                    monomial = "1"
                monomials.append(monomial)
        # "1" should be the first monomial. Note that order d > c > b > a means vector of monomials = [a, b, c, d].            
        return monomials[::-1]

    @staticmethod
    def sample_matrix(sample_points: list[int], degree: int) -> np.ndarray:
        """Construct a matrix from monomials evaluated at sample points."""
        n = len(sample_points[0])  # type: ignore # Number of variables based on the first sample point
        monomials = get_monomials(n, degree) # type: ignore
        matrix = np.zeros((len(sample_points), len(monomials)))

        for i, point in enumerate(sample_points):
            for j, monomial in enumerate(monomials):
                var_mapping = {f"λ_{k+1}": point[k] for k in range(n)} # type: ignore
                matrix[i, j] = eval(monomial, {}, var_mapping)
        return matrix

    @staticmethod
    def generate_modified_matrices(matrix):
        n = len(matrix)  # Size of the square matrix
        identity_row = [1] + [0] * (n - 1)  # Row to replace with

        modified_matrices = []
        determinants = []
        for i in range(n):
            # Create a copy of the original matrix
            modified_matrix = np.copy(matrix)
            # Replace the i-th row with the identity_row
            modified_matrix[i] = identity_row
            modified_matrices.append(modified_matrix)
            # Calculate the determinant of the modified matrix
            determinant = np.linalg.det(modified_matrix)
            determinants.append(determinant)
        
        return modified_matrices 

System #1

```qubit = 8
layer = 25
noise probabilities = [0.0001, 0.0001, 0.0001, 0.0001]
exact solution = -7.000000000000004
```


In [17]:
data_sys1 = [
    (4, 1, 0, -6.742247461826676),
    (12, 3, 8, -6.405455124800368),
    (12, 5, 24, -5.839787462274414),
    (4, 7, 8, -5.991097523966149),
    (4, 7, 24, -5.533783365921837),
    (28, 9, 24, -5.005206166353052),
    (4, 9, 56, -4.206331147604706),
    (44, 1, 24, -6.357848309361182),
    (44, 3, 24, -5.9516342852267075),
    (4, 13, 8, -5.328222900577986)
]

zne_sys1 = ZeroNoiseExtrapolation(dataPoints= data_sys1, degree= 2)
zne_sys1.getRichardsonZNE()

-6.821098781914534

System #2

```qubit = 8
layer = 25
noise probabilities = [0.000001, 0.000001, 0.000001, 0.000001]
exact solution = -7.000000000000004
```
